In [2]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from skimage.feature import hog
import xgboost as xgb
from sklearn.cluster import KMeans

def load_img(f, num_clusters=50, func="orb"):
    f = open(f)
    lines = f.readlines()
    imgs, lab = [], []
    if func=="orb":
        orb = cv2.ORB_create()
    elif func=="brisk":
        brisk = cv2.BRISK_create()
    # Perform k-means clustering on descriptors
    descriptors = []
    for i in range(len(lines)):
        line = lines[i].strip()  # Remove leading/trailing whitespaces
        fn, label = line.split(' ')
        label = int(label)
        if label in [0, 1, 2, 3, 4]:
            im1 = cv2.imread(fn)
            im1 = cv2.resize(im1, (256, 256))
            im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
            if func=="orb":
                kp, des = orb.detectAndCompute(im1, None)
            elif func=="brisk":
                kp, des = brisk.detectAndCompute(im1, None)
            elif func=="hog":
                des, _ = hog(im1, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys', visualize=True)
            if des is not None:
                descriptors.extend(des)
    descriptors = np.array(descriptors)
    if func=="hog":
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(descriptors.reshape(-1, 1))
    else:
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(descriptors)
    
    for i in range(len(lines)):
        line = lines[i].strip()  # Remove leading/trailing whitespaces
        fn, label = line.split(' ')
        label = int(label)
        if label in [0, 1, 2, 3, 4]:
            im1 = cv2.imread(fn)
            im1 = cv2.resize(im1, (256, 256))
            im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
            if func=="orb":
                kp, des = orb.detectAndCompute(im1, None)
            elif func=="brisk":
                kp, des = brisk.detectAndCompute(im1, None)
            elif func=="hog":
                des, _ = hog(im1, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys', visualize=True)
            features = np.zeros(num_clusters)
            if des is not None:
                if func=="hog":
                    labels = kmeans.predict(des.reshape(-1, 1))
                else:
                    labels = kmeans.predict(des)
                features = np.bincount(labels, minlength=num_clusters)
            imgs.append(features)
            lab.append(int(label))

    imgs = np.asarray(imgs)
    lab = np.asarray(lab, np.int32)
    return imgs, lab

classifiers = {
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Support Vector Machine': SVC(),
        'XGBoost': xgb.XGBClassifier()  # Adding XGBoost here
    }

for name, clf in classifiers.items():
    # Load training and testing data
    for extract_func in ['brisk','orb','hog']:
        x, y = load_img('train.txt',func=extract_func)
        tx, ty = load_img('test.txt',func=extract_func)
        clf.fit(x, y)
        pred = clf.predict(tx)
        acc = accuracy_score(ty, pred)
        f1 = f1_score(ty, pred, average='weighted')
        print(f"{name} {extract_func} - Accuracy: {acc}, F1-Score: {f1}")

c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


K-Nearest Neighbors brisk - Accuracy: 0.4, F1-Score: 0.3333333333333333


c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


K-Nearest Neighbors orb - Accuracy: 0.4, F1-Score: 0.3333333333333333


c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

XGBoost brisk - Accuracy: 0.6, F1-Score: 0.5333333333333333


c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


XGBoost orb - Accuracy: 0.8, F1-Score: 0.7333333333333333


c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


XGBoost hog - Accuracy: 0.0, F1-Score: 0.0


In [3]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

def load_img(f, target_size=(224, 224)):
    try:
        f = open(f)
        lines = f.readlines()
        imgs, lab = [], []
        for i in range(len(lines)):
            line = lines[i].strip()  # Remove leading/trailing whitespaces
            fn, label = line.split(' ')
            label = int(label)
            if label in [0, 1, 2, 3, 4]:
                img = image.load_img(fn, target_size=target_size)
                img = image.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = preprocess_input(img)
                imgs.append(img)
                lab.append(label)

        imgs = np.vstack(imgs)
        lab = np.array(lab)
        return imgs, lab
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Load training and testing data
x, y = load_img('train.txt')
tx, ty = load_img('test.txt')

if x is not None and y is not None and tx is not None and ty is not None:
    # Load pre-trained ResNet50 model without the top (classification) layer
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    # Extract features using the pre-trained ResNet50 model
    x_features = base_model.predict(x)
    tx_features = base_model.predict(tx)

    classifiers = {
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Support Vector Machine': SVC(),
        'XGBoost': xgb.XGBClassifier()  # Adding XGBoost here
    }

    for name, clf in classifiers.items():
        clf.fit(x_features, y)
        pred = clf.predict(tx_features)
        acc = accuracy_score(ty, pred)
        f1 = f1_score(ty, pred, average='weighted')
        print(f"{name} - Accuracy: {acc}, F1-Score: {f1}")
else:
    print("Error loading data. Please check the file paths and formats.")





1/1 [==============================] - 0s 114ms/step
K-Nearest Neighbors - Accuracy: 0.8, F1-Score: 0.7333333333333333
Support Vector Machine - Accuracy: 0.8, F1-Score: 0.7333333333333333
XGBoost - Accuracy: 0.8, F1-Score: 0.7333333333333333
